In [1]:
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')

In [3]:
from tqdm.auto import tqdm

In [1]:
# --- Imports ---
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct, Filter, FieldCondition, MatchValue
from tqdm import tqdm
import numpy as np



/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:

# --- Sanity checks (expects you already have these) ---
# - documents: list of dicts with keys: id, question, text, course
# - ground_truth: list of dicts with keys: question, course, document (the id of the correct doc)
assert isinstance(documents, list) and len(documents) > 0, "Expected non-empty `documents` list."
assert isinstance(ground_truth, list) and len(ground_truth) > 0, "Expected non-empty `ground_truth` list."


In [7]:

# --- Helper: normalize course values (avoids case/whitespace mismatches) ---
def norm_course(x):
    return (x or "").strip().lower()



In [8]:
# Ensure every doc has a normalized course for filtering
for d in documents:
    d["course_norm"] = norm_course(d.get("course", ""))



In [9]:
# --- Build texts: question + answer ---
texts_qA = [f"{d['question']} {d['text']}" for d in documents]


In [2]:

# --- Load embedding model ---
model_handle = "jinaai/jina-embeddings-v2-small-en"
model = SentenceTransformer(model_handle, trust_remote_code=True)


NameError: name 'SentenceTransformer' is not defined

In [1]:

# --- Encode corpus (normalize for cosine similarity) ---
embs = model.encode(texts_qA, normalize_embeddings=True)
embs = np.asarray(embs)
vector_size = embs.shape[1]  # should be 512 for this model

NameError: name 'model' is not defined

In [5]:
# Model (Q+A embeddings)
model = SentenceTransformer("jinaai/jina-embeddings-v2-small-en", trust_remote_code=True)

# Qdrant client — pick ONE of these:
# If you have a Qdrant server running:
# client = QdrantClient(host="localhost", port=6333)

# If you DON'T have a server, use embedded/in-memory:
client = QdrantClient(":memory:")  # or client = QdrantClient(path=":memory:")

# Create/recreate collection (512-dim for this model, cosine distance)
COLL = "qa_qdrant_jina_small"
client.recreate_collection(
    collection_name=COLL,
    vectors_config=VectorParams(size=512, distance=Distance.COSINE),
)

/tmp/ipykernel_68392/474015915.py:13: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [6]:
points = [
    PointStruct(
        id=str(doc["id"]),
        vector=embs[i].tolist(),
        payload={"id": str(doc["id"]), "course": doc["course"], "question": doc["question"], "text": doc["text"]},
    )
    for i, doc in enumerate(documents)
]
client.upsert(collection_name=COLL, points=points)

NameError: name 'embs' is not defined